# What is Flask?

A lightweight Python web framework [Flask][flask]

- Very little boilerplate
- Many choices for storage back-ends, but none included in Flask itself

[flask]: http://flask.pocoo.org/


# ReST: Representational State Transfer

Everyone *says* they're restful, but...

REST has a few attributes that make it truly RESTful:

- URLs represent *resources* (i.e. nouns), not *actions* (i.e. verbs)
    - a POST to `/mailingList/createUser` is **not** RESTful
- Resources may have one or more *representations* (formats transferred over the web)
    - HTML
    - XML
    - JSON
- HTTP verbs are used to represent operations on resources
    - GET - safe & idempotent, give me the representation for a resource
    - PUT - idempotent, replace a resource at a given URL
    - PATCH - partial update to a resource at a given URL
    - DELETE - idempotent, delete a resource at a given URL
    - POST - do something else (frequently create)
- Representations must communicate their relationships to other resources via *hypertext*
    - Generally, this means representations have URLs to relate to other resources, *not* just IDs
    - Requiring knowledge of URL layout is not RESTful
    - "Hypertext as the engine of application state" or HATEOAS
    - We'll use a tiny part of a HATEOAS standard known as HAL

# Getting started - our first Flask API

- Single endpoint/resource: the root (/)
- Single representation: `{"hello": "world"}`
- Single operation: GET
- A flask **view** is a function that maps to a URL

In [ ]:
!pip install flask

Normally, we would run flask by executing the following in the shell:


```bash
$ FLASK_APP=intro_flask.app1 FLASK_ENV=development flask run
```

Since we're using the notebook, here's a little program to run a subprocess in a thread:

In [ ]:
!which flask

In [3]:
import os, sys, threading, subprocess

def output_thread(proc):
    for line in proc.stdout:
        print(line.decode('utf-8'), end='')
    print('Exiting output thread')

def run_flask_app(app_name):
    proc = subprocess.Popen(
        # [sys.executable, 'flask', 'run'],
        ['flask', 'run', '--no-reload'],
        env={
            **os.environ, 
            'FLASK_APP': app_name,
            'FLASK_ENV': 'development',
        },
        stderr=subprocess.STDOUT,
        stdout=subprocess.PIPE
    )
    # Wait for the port to bind
    for line in proc.stdout:
        line = line.decode('utf-8')
        print(line, end='')
        if ' * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)' in line:
            break
    else:
        print('== Error starting server ==')
        return None
    thd = threading.Thread(target=output_thread, args=(proc,))
    thd.setDaemon(True)
    thd.start()
    return proc


# The task: build a REST API for a blog in Flask

- What are our resources?
- What are the operations on the resources?

In [4]:
%%file data/flask-examples/app1.py
import flask

app = flask.Flask(__name__)

@app.route('/')
def get_root():
    return flask.jsonify(hello='world')

Overwriting data/flask-examples/app1.py


In [5]:
sp = run_flask_app('data.flask-examples.app1')

 * Serving Flask app "data.flask-examples.app1"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [6]:
import requests
resp = requests.get('http://localhost:5000')
resp.json()

127.0.0.1 - - [22/Mar/2019 09:24:14] "GET / HTTP/1.1" 200 -


{'hello': 'world'}

In [7]:
!curl localhost:5000

127.0.0.1 - - [22/Mar/2019 09:24:18] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}
127.0.0.1 - - [22/Mar/2019 09:24:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2019 09:24:27] "GET /favicon.ico HTTP/1.1" 404 -


In [8]:
sp.kill()

Exiting output thread


# One more helper: run an app in a context manager

In [9]:
import time, contextlib

@contextlib.contextmanager
def running_app(app_name):
    proc = run_flask_app(app_name)
    try:
        yield proc
    finally:
        proc.kill()        

In [10]:
with running_app('data.flask-examples.app1'):
    !curl localhost:5000

 * Serving Flask app "data.flask-examples.app1"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:25:54] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}
Exiting output thread


# Handling url parameters

In [11]:
%%file data/flask-examples/app2.py
import flask

app = flask.Flask(__name__)

@app.route('/')
def get_root():
    return flask.jsonify(hello='world')

@app.route('/<name>')
def get_name(name):
    return flask.jsonify(hello=name)

Overwriting data/flask-examples/app2.py


In [12]:
with running_app('data.flask-examples.app2'):
    !curl localhost:5000
    !curl localhost:5000/rick
    !curl localhost:5000/starbucks

 * Serving Flask app "data.flask-examples.app2"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:26:44] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}
127.0.0.1 - - [22/Mar/2019 09:26:44] "GET /rick HTTP/1.1" 200 -
{
  "hello": "rick"
}
127.0.0.1 - - [22/Mar/2019 09:26:44] "GET /starbucks HTTP/1.1" 200 -
{
  "hello": "starbucks"
}
Exiting output thread


# Handling JSON data

In [13]:
%%file data/flask-examples/app3.py
import flask

app = flask.Flask(__name__)

state = {'name': 'world'}

@app.route('/')
def get_root():
    return flask.jsonify(hello=state['name'])

@app.route('/', methods=['PUT'])
def set_name():
    body = flask.request.json  # resolves to None if no valid JSON Content-Type header
    state['name'] = body['name']
    return flask.jsonify(name=state['name'])

Overwriting data/flask-examples/app3.py


In [18]:
with running_app('data.flask-examples.app3'):
    !curl localhost:5000
    !curl -XPUT -H 'Content-Type: application/json' -d '{"name": "Rick"}' localhost:5000 
    !curl localhost:5000

 * Serving Flask app "data.flask-examples.app3"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:31:21] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}
127.0.0.1 - - [22/Mar/2019 09:31:21] "PUT / HTTP/1.1" 200 -
{
  "name": "Rick"
}
127.0.0.1 - - [22/Mar/2019 09:31:21] "GET / HTTP/1.1" 200 -
{
  "hello": "Rick"
}
Exiting output thread


In [19]:
with running_app('data.flask-examples.app3'):
    print(requests.get('http://localhost:5000').json())
    !curl -XPUT -H 'Content-Type: application/json' -d '{"name": "Rick"}' localhost:5000 
    !curl localhost:5000

 * Serving Flask app "data.flask-examples.app3"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:32:53] "GET / HTTP/1.1" 200 -
{'hello': 'world'}
127.0.0.1 - - [22/Mar/2019 09:32:53] "PUT / HTTP/1.1" 200 -
{
  "name": "Rick"
}
127.0.0.1 - - [22/Mar/2019 09:32:53] "GET / HTTP/1.1" 200 -
{
  "hello": "Rick"
}
Exiting output thread


# Using auth data (basic) & URL generation

In [28]:
%%file data/flask-examples/app4.py
import flask

app = flask.Flask(__name__)

@app.route('/profile/<username>')
def get_profile(username):
    return flask.jsonify(username=username)

@app.route('/userinfo')
def get_userinfo():
    username = flask.request.authorization['username']
    profile_url = flask.url_for(
        'get_profile', username=username, 
        _external=True
    )
    return flask.jsonify(
        _links={'profile': profile_url},
        username=username,
        password=flask.request.authorization['password'] # don't do this in production, obviously
    )

Overwriting data/flask-examples/app4.py


In [29]:
with running_app('data.flask-examples.app4'):
    !curl rick:secret@localhost:5000/userinfo

 * Serving Flask app "data.flask-examples.app4"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:37:38] "GET /userinfo HTTP/1.1" 200 -
{
  "_links": {
    "profile": "http://localhost:5000/profile/rick"
  }, 
  "password": "secret", 
  "username": "rick"
}
Exiting output thread


# Building the blog post API

## Resource structure

```
{
    _links: {self: <link_to_post>},
    postedDate: ...,
    authorName: ...,
    title: ...,
    body: ...,
}
```

## URL structure

- / 
    - GET: return `{_links: {posts: /post}}`
- /post
    - GET: return list of posts
    - POST: create and return a post
- `/post/<post_id>` - return a single post
    - GET: return the post
    - PUT: update the post
    - DELETE: delete the post



In [30]:
%%file data/flask-examples/app5.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Flask, jsonify, request, abort


app = Flask(__name__)

state = {
    'posts': {
        uuid4().hex: {
            'postedDate': datetime.utcnow(),
            'authorName': 'rick',
            'title': 'first!',
            'body': 'First post!'
        }
    }
}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('get_posts')})

@app.route('/post')
def get_posts():
    post_links = [
        url_for('get_post', id=id) 
        for id in state['posts']
    ]
    return jsonify(
        _links={'self': url_for('get_posts')},
        data=[
            dict(_links=dict(self=link)) 
            for link in post_links
        ]
    )

@app.route('/post/<id>')
def get_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    return jsonify(
        _links={'self': url_for('get_post', id=id)},
        postedDate=post['postedDate'].isoformat(),
        authorName=post['authorName'],
        title=post['title'],
        body=post['body']
    )

Overwriting data/flask-examples/app5.py


In [31]:
with running_app('data.flask-examples.app5'):
    !curl localhost:5000
    !curl localhost:5000/post
    print('Got post, now need to return...')

 * Serving Flask app "data.flask-examples.app5"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:42:19] "GET / HTTP/1.1" 200 -
{
  "_links": {
    "posts": "http://localhost:5000/post"
  }
}
127.0.0.1 - - [22/Mar/2019 09:42:19] "GET /post HTTP/1.1" 200 -
{
  "_links": {
    "self": "http://localhost:5000/post"
  }, 
  "data": [
    {
      "_links": {
        "self": "http://localhost:5000/post/e0879c2aed484fddba6892775661e4d6"
      }
    }
  ]
}
Got post, now need to return...
Exiting output thread


In [32]:
import requests

with running_app('data.flask-examples.app5'):
    root = requests.get('http://localhost:5000').json()
    print('root is', root)
    posts = requests.get(root['_links']['posts']).json()
    print('posts is', posts)
    post = requests.get(posts['data'][0]['_links']['self']).json()
    print('post is', post)


 * Serving Flask app "data.flask-examples.app5"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:43:23] "GET / HTTP/1.1" 200 -
root is {'_links': {'posts': 'http://localhost:5000/post'}}
127.0.0.1 - - [22/Mar/2019 09:43:23] "GET /post HTTP/1.1" 200 -
posts is {'_links': {'self': 'http://localhost:5000/post'}, 'data': [{'_links': {'self': 'http://localhost:5000/post/15ff3c27f584480c9cf1f2dd0638b7e4'}}]}
127.0.0.1 - - [22/Mar/2019 09:43:23] "GET /post/15ff3c27f584480c9cf1f2dd0638b7e4 HTTP/1.1" 200 -
post is {'_links': {'self': 'http://localhost:5000/post/15ff3c27f584480c9cf1f2dd0638b7e4'}, 'authorName': 'rick', 'body': 'First post!', 'postedDate': '2019-03-22T16:43:23.213497', 'title': 'first!'}
Exiting output thread


In [33]:
post

{'_links': {'self': 'http://localhost:5000/post/15ff3c27f584480c9cf1f2dd0638b7e4'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2019-03-22T16:43:23.213497',
 'title': 'first!'}

# Aside: `glom` can be handy for consuming deeply nested dicts like this

In [34]:
!pip install glom

  Using cached https://files.pythonhosted.org/packages/87/ee/91abf3409fce15e087d20e831bc7a26d806acaa133e5ed9fd65d12e3da82/glom-19.2.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/32/e8/f1fcb6ba6b70d2582e7d5cb4710b4ac39f6da1976286f5306be79913d545/boltons-19.1.0-py2.py3-none-any.whl
  Running setup.py install for face ... done
  Running setup.py install for glom ... done
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [35]:
from glom import glom

In [36]:
import requests

with running_app('data.flask-examples.app5'):
    root = requests.get('http://localhost:5000').json()
    posts = requests.get(glom(root, '_links.posts')).json()
    post = requests.get(glom(posts, 'data.0._links.self')).json()
post

 * Serving Flask app "data.flask-examples.app5"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:44:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2019 09:44:33] "GET /post HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2019 09:44:33] "GET /post/34e661c59ff54879903382df589432fd HTTP/1.1" 200 -


{'_links': {'self': 'http://localhost:5000/post/34e661c59ff54879903382df589432fd'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2019-03-22T16:44:33.265604',
 'title': 'first!'}

Exiting output thread


## Modifying posts

In [37]:
%%file data/flask-examples/app6.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Flask, jsonify, request, abort


app = Flask(__name__)

state = {
    'posts': { }
#         uuid4().hex: {
#             'postedDate': datetime.utcnow(),
#             'authorName': 'rick',
#             'title': 'first!',
#             'body': 'First post!'
#         }
#     }
}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('get_posts')})

@app.route('/post')
def get_posts():
    post_links = [url_for('get_post', id=id) for id in state['posts']]
    return jsonify(
        _links={'self': url_for('get_posts')},
        data=[dict(_links=dict(self=link)) for link in post_links])

@app.route('/post', methods=['POST'])
def create_post():
    post_id = uuid4().hex
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json
    }
    state['posts'][post_id] = post
    result = jsonify_post(post_id, post)
    result.headers['Location'] = url_for('get_post', id=post_id)
    return result, 201

@app.route('/post/<id>')
def get_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    return jsonify_post(id, post)

@app.route('/post/<id>', methods=['PUT'])
def update_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json
    }
    state['posts'][id] = post
    return jsonify_post(id, post)

@app.route('/post/<id>', methods=['DELETE'])
def delete_post(id):
    state['posts'].pop(id)
    return '', 204

def jsonify_post(id, post, **kwargs):
    return jsonify(
        _links={'self': url_for('get_post', id=id)},
        postedDate=post['postedDate'].isoformat(),
        authorName=post['authorName'],
        title=post['title'],
        body=post['body']
    )
    

Overwriting data/flask-examples/app6.py


In [38]:
from pprint import pprint

sess = requests.Session()
sess.auth = ('rick', 'password')
sess.headers['Content-Type'] = 'application/json'

with running_app('data.flask-examples.app6'):
    root = sess.get('http://localhost:5000').json()
    posts_url = glom(root, '_links.posts')
    resp = sess.post(posts_url, json=dict(title='First!', body="First post!"))
    pprint(resp.headers['Location'])
    input('Press enter to continue')
    post1 = resp.json()
    pprint(post1)
    input('Press enter to continue')
    sess.post(posts_url, json=dict(title='Second!', body="Second post!"))
    sess.post(posts_url, json=dict(title='Third!', body="Another post!"))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')
    resp = sess.delete(glom(post1, '_links.self'))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')


 * Serving Flask app "data.flask-examples.app6"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:47:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2019 09:47:45] "POST /post HTTP/1.1" 201 -
'http://localhost:5000/post/ba694badd2e14f3ca828cf851f7fbf31'
Press enter to continue
{'_links': {'self': 'http://localhost:5000/post/ba694badd2e14f3ca828cf851f7fbf31'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2019-03-22T16:47:45.746058',
 'title': 'First!'}
Press enter to continue
127.0.0.1 - - [22/Mar/2019 09:48:44] "POST /post HTTP/1.1" 201 -
127.0.0.1 - - [22/Mar/2019 09:48:44] "POST /post HTTP/1.1" 201 -
127.0.0.1 - - [22/Mar/2019 09:48:44] "GET /post HTTP/1.1" 200 -
{'_links': {'self': 'http://localhost:5000/post'},
 'data': [{'_links': {'self': 'http://localhost:5000/post/ba694badd2e14f3ca828cf851f7fbf31'}},
          {'_links': {'self': 'http://localhost:5000/post/5afb301a669842b59

# Organizing code into blueprints for reusability and maintenance

In [45]:
%%file data/flask-examples/app7.py
from flask import Flask, jsonify, url_for

from . import app7_posts
from .app7_util import url_for

app = Flask(__name__)

app.register_blueprint(app7_posts.mod, url_prefix='/post')

@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('posts.get_root')})

Overwriting data/flask-examples/app7.py


In [40]:
%%file data/flask-examples/app7_util.py
import flask

state = {'posts': {}}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

Overwriting data/flask-examples/app7_util.py


In [43]:
%%file data/flask-examples/app7_posts.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Blueprint, jsonify, request, abort

from .app7_util import state, url_for

mod = Blueprint('posts', __name__)

@mod.route('')
def get_root():
    post_links = [url_for('.get_post', id=id) for id in state['posts']]
    return jsonify(
        _links={
            'self': url_for('.get_root'),
            'home': url_for('get_root'),
        },
        data=[dict(_links=dict(self=link)) for link in post_links])

@mod.route('', methods=['POST'])
def create_post():
    post_id = uuid4().hex
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json
    }
    state['posts'][post_id] = post
    result = jsonify_post(post_id, post)
    result.headers['Location'] = url_for('.get_post', id=post_id)
    return result

@mod.route('<id>')
def get_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    return jsonify_posts(id, post)

@mod.route('<id>', methods=['PUT'])
def update_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json
    }
    state['posts'][id] = post
    return jsonify_post(id, post)

@mod.route('<id>', methods=['DELETE'])
def delete_post(id):
    state['posts'].pop(id)
    return '', 204

def jsonify_post(id, post, **kwargs):
    return jsonify(
        _links={'self': url_for('.get_post', id=id)},
        postedDate=post['postedDate'].isoformat(),
        authorName=post['authorName'],
        title=post['title'],
        body=post['body']
    )

Overwriting data/flask-examples/app7_posts.py


In [44]:
from pprint import pprint

sess = requests.Session()
sess.auth = ('rick', 'password')
sess.headers['Content-Type'] = 'application/json'

with running_app('data.flask-examples.app7'):
    root = sess.get('http://localhost:5000').json()
    posts_url = glom(root, '_links.posts')
    resp = sess.post(posts_url, json=dict(title='First!', body="First post!"))
    pprint(resp.headers['Location'])
    input('Press enter to continue')
    post1 = resp.json()
    pprint(post1)
    input('Press enter to continue')
    sess.post(posts_url, json=dict(title='Second!', body="Second post!"))
    sess.post(posts_url, json=dict(title='Third!', body="Another post!"))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')
    resp = sess.delete(glom(post1, '_links.self'))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')


 * Serving Flask app "data.flask-examples.app7"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 09:57:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2019 09:57:30] "POST /post HTTP/1.1" 200 -
'http://localhost:5000/post/e1ebf41813584a02b7296064d43d1f99'
Press enter to continue
{'_links': {'self': 'http://localhost:5000/post/e1ebf41813584a02b7296064d43d1f99'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2019-03-22T16:57:30.048764',
 'title': 'First!'}
Press enter to continue
127.0.0.1 - - [22/Mar/2019 09:57:34] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2019 09:57:34] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2019 09:57:34] "GET /post HTTP/1.1" 200 -
{'_links': {'home': 'http://localhost:5000/',
            'self': 'http://localhost:5000/post'},
 'data': [{'_links': {'self': 'http://localhost:5000/post/e1ebf41813584a02b7296064d43d1f99'}},
          {'_links': {'self':

# Lab

Open the [Flask APIs lab][flask-api-lab]

[flask-api-lab]: ./flask-api-lab.ipynb